<a href="https://colab.research.google.com/github/namwootree/Certificate/blob/main/Big_Data_Analytics/3_%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_%EA%B8%B0%EC%82%AC_%EC%8B%A4%EA%B8%B0_%EB%8C%80%EB%B9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 유형

다음은
Boston Housing 데이터셋이다 . crim 항목의 상위에서 10 번째 값 즉 , 상위 10 번째 값
중에서 가장 작은 값 으로 상위 10 개의 값을 변환하고 age 80 이상인 값에 대하여 crim 평균을
구하시오

In [71]:
import pandas as pd

df = pd.read_csv('/content/boston_housing.csv')

# print(df.shape) # 560 / 14
# print(df.columns)
# print(df.info())

df_top_10 = df.sort_values(by='crim', ascending=False).head(10)

# print(df_top_10.shape) # 10 /14
# print(df_top_10['crim'].min()) # 25.9406

cond1 = (df['crim']>=df_top_10['crim'].min())

df.loc[cond1, 'new_crim'] = df_top_10['crim'].min()
df.loc[~cond1, 'new_crim'] = df['crim']

cond2 = (df['age']>=80)
df1 = df.loc[cond2]
print(df1['new_crim'].mean()) # 5.759387

5.759386625


주어진
데이터의 첫 번째 행부터 순서대로 80% 까지의 데이터를 훈련 데이터로 추출 후
total_bedrooms ’ 변수의 결측값 ( 을 total_bedrooms ’ 변수의 중앙값으로 대체하고 대체 전의
total_bedrooms 변수 표준편차 값의 차이의 절댓값을 구하시오

In [96]:
import pandas as pd

df = pd.read_csv('/content/california_housing.csv')

# print(df.shape) # 20640 / 10
# print(df.columns)
# print(df.info()) # total_bedrooms 결측치 존재

df_80 = df.iloc[:int(20640 * 0.8)]

# print(df_80.shape) # 16512 / 10

std1 = df_80['total_bedrooms'].std()

df_80['total_bedrooms'].fillna(df_80['total_bedrooms'].median(), inplace=True)

# print(df_80.info())

std2 = df_80['total_bedrooms'].std()

print(std1-std2) # 1.9751472916456692

1.9751472916456692


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


다음은
Insurance 데이터셋이다 . charges 항목의 이상값의 합을 구하시오
(
이상값은 평균에서 1.5 표준편차 이상인 값

In [108]:
import pandas as pd

df = pd.read_csv('/content/insurance.csv')

# print(df.shape) # 1338 / 7
# print(df.columns)
# print(df.info())

out1 = df['charges'].mean() + (1.5 * df['charges'].std())
# print(out1) # 31435.43912018226

out2 = df['charges'].mean() - (1.5 * df['charges'].std())
# print(out2) # -4894.594589899745

cond1 = (df['charges']>=out1)
cond2 = (df['charges']<=out2)
df1 = df.loc[cond1|cond2]
print(df1['charges'].sum()) # 6421430.0206699995

6421430.0206699995


# 2 유형

아래
E Commerce Shipping Data 의 train set 을 참조하여 고객이 주문한 물품의 정시 도착
여부를 예측하시오 . (ID 와 예측치를 csv 파일로 저장하여 제출하시오

In [137]:
import pandas as pd

x = pd.read_csv('/content/X_train.csv')
y = pd.read_csv('/content/y_train.csv')

# print(x.shape) # 8799 / 11
# print(y.shape) # 8799 / 2

# print(x.info())
# print(y.info())

# print(x.columns)
# print(y.columns)

x_train = x.drop(columns=['ID', 'Gender'])
x_train = pd.get_dummies(x_train)
y_train = y.drop(columns='ID')

# print(x_train.columns)
# print(y_train.columns)

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

pipe_list = [('model', RandomForestClassifier())]

pipe_model = Pipeline(pipe_list)

param_grid = {
    'model__min_samples_split':range(5, 7),
    'model__n_estimators':[100, 150, 200],
    'model__class_weight':[None, 'balanced']
}

grid_model = GridSearchCV(pipe_model, param_grid=param_grid,
                          cv=3, n_jobs=-1, scoring='f1')

grid_model.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('model', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'model__class_weight': [None, 'balanced'],
                         'model__min_samples_split': range(5, 7),
                         'model__n_estimators': [100, 150, 200]},
             scoring='f1')

In [138]:
best_model = grid_model.best_estimator_
print(best_model)

Pipeline(steps=[('model',
                 RandomForestClassifier(min_samples_split=5,
                                        n_estimators=200))])


In [139]:
pred_y_train = best_model.predict(x_train)
print(classification_report(y_train, pred_y_train))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3548
           1       1.00      1.00      1.00      5251

    accuracy                           1.00      8799
   macro avg       1.00      1.00      1.00      8799
weighted avg       1.00      1.00      1.00      8799



In [144]:
x_test = pd.read_csv('/content/X_test.csv')
# print(x.shape) # 8799 / 11

list_id = list(x_test['ID'])

x_test = x_test.drop(columns=['ID', 'Gender'])
x_test = pd.get_dummies(x_test)

pred_y_test = best_model.predict(x_test)

list_reached = list(pred_y_test)

result_dict = {
    'ID':list_id,
    'Reached.on.Time_Y.N':list_reached
    }

result_df = pd.DataFrame(result_dict)

result_df.to_csv('20220624.csv', index=False)